
### Python list comprehension

In [7]:
from gurobipy import *

In [5]:
# List comprehension
squares = [i**2 for i in range(6)]

In [6]:
# Generator expression
SumSquares = sum(i**2 for i in range(6))

In [3]:
pairs = [(i, j) for j in range(4) for i in range(j)]

In [4]:
Nonsquares = [i for i in range(10) if i not in squares]

The Gurobi interface takes advantage of lists, list comprehension and generator expressions:

A Gurobi class to store lists of tuples

In [8]:
Cities = ['A', 'B', 'C', 'D']
Routes = tuplelist([('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'D'), ('C', 'D')])

What makes it special: select statement for efficient filtering

In [9]:
for c in Cities:
    print(Routes.select(c, '*'))

<gurobi.tuplelist (2 tuples, 2 values each):
 ( A , B )
 ( A , C )
>
<gurobi.tuplelist (2 tuples, 2 values each):
 ( B , C )
 ( B , D )
>
<gurobi.tuplelist (1 tuples, 2 values each):
 ( C , D )
>
[]


The tuplelist is indexed to make select() efficient

### Indexed variables: Model.addVars

In [10]:
# Using integers
m = Model("model 1")
x = m.addVars(2, 3, name = 'x')
x

Set parameter Username
Set parameter LicenseID to value 2588857
Academic license - for non-commercial use only - expires 2025-11-22


{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>}

In [12]:
# Using lists of scalars
m = Model("model 2")
y = m.addVars(Cities, Cities, name = 'y')
y

{('A', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('A', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('A', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('A', 'D'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'D'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'D'): <gurobi.Var *Awaiting Model Update*>,
 ('D', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('D', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('D', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('D', 'D'): <gurobi.Var *Awaiting Model Update*>}

In [13]:
# Using a tuplelist
z = m.addVars(Routes, name = 'z')
z

{('A', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('A', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'D'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'D'): <gurobi.Var *Awaiting Model Update*>}

In [14]:
# Using a generator expression(!)
w = m.addVars((i for i in range(5) if i != 2), name = 'w')
w

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}